# Desafío 2 - Introducción a Big Data

**Nombre:** Julio Valdés
<br>
**Generación:** GC

### Ejercicio 1: Ingesta de datos semiestructurados

In [6]:
import requests
import json

url = 'https://www.balldontlie.io/api/v1/games'

params = dict(
    per_page = 100
)

data = requests.get(url=url, params=params).json()
data['data'][0]

{'id': 47179,
 'date': '2019-01-30T00:00:00.000Z',
 'home_team': {'id': 2,
  'abbreviation': 'BOS',
  'city': 'Boston',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Boston Celtics',
  'name': 'Celtics'},
 'home_team_score': 126,
 'period': 4,
 'postseason': False,
 'season': 2018,
 'status': 'Final',
 'time': ' ',
 'visitor_team': {'id': 4,
  'abbreviation': 'CHA',
  'city': 'Charlotte',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Charlotte Hornets',
  'name': 'Hornets'},
 'visitor_team_score': 94}

#### Comentarios:
* Cada juego tiene un identificador llamado 'id'.
* Cada home_team tiene un identificador llamado 'id'.
* Cada visitor_team tiene un identificador llamado 'id'

### Ejercicio 2: Organización de los datos

In [7]:
import pandas as pd

get_row = lambda x: [x['season'], x['period'],
    x['home_team']['full_name'], x['home_team']['conference'], x['home_team']['division'],
    x['visitor_team']['full_name'], x['visitor_team']['conference'], x['visitor_team']['division'],
    x['home_team_score'], x['visitor_team_score'],
    x['home_team_score'] - x['visitor_team_score'], x['visitor_team_score'] - x['home_team_score']]
df_data = list(map(get_row, data['data']))
df = pd.DataFrame(df_data, columns = [
    'season', 'period',
    'home', 'home_conference', 'home_division',
    'visitor', 'visitor_conference', 'visitor_division',
    'home_score', 'visitor_score', 'home_differential', 'visitor_differential'])
df.head(5)

,season,period,home,home_conference,home_division,visitor,visitor_conference,visitor_division,home_score,visitor_score,home_differential,visitor_differential
0,2018,4,Boston Celtics,East,Atlantic,Charlotte Hornets,East,Southeast,126,94,32,-32
1,2018,4,Boston Celtics,East,Atlantic,LA Clippers,West,Pacific,112,123,-11,11
2,2018,4,Philadelphia 76ers,East,Atlantic,Denver Nuggets,West,Northwest,117,110,7,-7
3,2018,4,Washington Wizards,East,Southeast,Cleveland Cavaliers,East,Central,119,106,13,-13
4,2018,4,Sacramento Kings,West,Pacific,Miami Heat,East,Southeast,102,96,6,-6


### Ejercicio 3: El efecto de jugar de local

In [9]:
import numpy as np
df['home_win'] = np.where(df['home_score'] > df['visitor_score'], 1, 0)
df['visitor_win'] = np.where(df['home_score'] < df['visitor_score'], 1, 0)

df_home_win = df.groupby(['home']).sum().reset_index()[['home', 'home_win']]

In [10]:
print("Los primeros 5 equipos en cuanto a desempeño por jugar de local son:")
df_home_win.sort_values(by = ['home_win', 'home'], ascending = [False, True]).head(5)

Los primeros 5 equipos en cuanto a desempeño por jugar de local son:


,home,home_win
8,Detroit Pistons,4
11,Indiana Pacers,4
20,Orlando Magic,4
21,Philadelphia 76ers,4
28,Washington Wizards,4


In [11]:
print("Los últimos 5 equipos en cuanto a desempeño por jugar de local son:")
df_home_win.sort_values(by = ['home_win', 'home'], ascending = [False, True]).tail(5)

Los últimos 5 equipos en cuanto a desempeño por jugar de local son:


,home,home_win
14,Miami Heat,1
3,Charlotte Hornets,0
18,New York Knicks,0
19,Oklahoma City Thunder,0
22,Phoenix Suns,0


In [13]:
df_visitor_win = df.groupby(['visitor']).sum().reset_index()[['visitor', 'visitor_win']]
print("Los primeros 5 equipos en cuanto a desempeño por jugar de visitante son:")
df_visitor_win.sort_values(by = ['visitor_win', 'visitor'], ascending = [False, True]).head(5)

Los primeros 5 equipos en cuanto a desempeño por jugar de visitante son:


,visitor,visitor_win
21,Orlando Magic,4
12,LA Clippers,3
1,Boston Celtics,2
2,Brooklyn Nets,2
3,Charlotte Hornets,2


In [14]:
print("Los últimos 5 equipos en cuanto a desempeño por jugar de visitante son:")
df_visitor_win.sort_values(by = ['visitor_win', 'visitor'], ascending = [False, True]).tail(5)

Los últimos 5 equipos en cuanto a desempeño por jugar de visitante son:


,visitor,visitor_win
14,Memphis Grizzlies,0
17,Minnesota Timberwolves,0
19,New York Knicks,0
23,Phoenix Suns,0
25,Sacramento Kings,0


### Ejercicio 4: Obteniendo el porcentaje de ganar local y de visita

#### Genere un nuevo objeto que guarde el porcentaje de juegos ganados como local por equipo.

In [15]:
df_home_win['home_win_percent'] = df_home_win['home_win'].apply(lambda x: float(x/df.shape[0]))

#### Repita lo mismo para los juegos donde el equipo fue visitante.

In [16]:
df_visitor_win['visitor_win_percent'] = df_visitor_win['visitor_win'].apply(lambda x: float(x/df.shape[0]))

#### ¿Qué equipos tienen iguales chances de ganar como visitante o local?

In [17]:
df_win_percent = pd.merge(df_home_win, df_visitor_win, left_on = 'home', right_on = 'visitor')
df_win_percent[df_win_percent['home_win_percent'] == df_win_percent['visitor_win_percent']]

,home,home_win,home_win_percent,visitor,visitor_win,visitor_win_percent
4,Chicago Bulls,1,0.01,Chicago Bulls,1,0.01
10,Houston Rockets,2,0.02,Houston Rockets,2,0.02
15,Milwaukee Bucks,2,0.02,Milwaukee Bucks,2,0.02
18,New York Knicks,0,0.00,New York Knicks,0,0.00
20,Orlando Magic,4,0.04,Orlando Magic,4,0.04
22,Phoenix Suns,0,0.00,Phoenix Suns,0,0.00
23,Portland Trail Blazers,2,0.02,Portland Trail Blazers,2,0.02
